# Indian House Price Prediction Model

**A simplified, production-ready ML pipeline for house price prediction**

- Clean data preprocessing
- Feature engineering
- Random Forest model training
- Model evaluation
- Model saving with joblib

## 1. Import Libraries

In [1]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

print("All libraries imported successfully!")

All libraries imported successfully!


## 2. Load Data

For Google Colab: Change the path to `"india_housing_prices.csv"`

In [ ]:
# Load dataset
df = pd.read_csv(r"")

print(f"Dataset loaded: {df.shape[0]} rows, {df.shape[1]} columns")
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Data_Science_Programme\\Machine_learning_models\\Indian_House_price_prediction\\india_housing_prices.csv'

## 3. Data Preprocessing

In [ ]:
# Display all columns
print("All columns in dataset:")
print(df.columns.tolist())
print(f"\nDataset info:")
print(df.info())

# Drop unnecessary columns
# ID - not useful, Price_per_SqFt - can be derived, Age_of_Property - duplicate of Year_Built
columns_to_drop = ['ID', 'Price_per_SqFt', 'Age_of_Property']
df = df.drop(columns=columns_to_drop, errors='ignore')

# Check for missing values
print("\nMissing values:")
missing = df.isnull().sum()
print(missing[missing > 0] if missing.sum() > 0 else "No missing values!")
print(f"\nDataset shape after dropping columns: {df.shape}")

## 4. Feature Engineering

In [ ]:
# Create Property_Age feature (if not already present)
CURRENT_YEAR = 2025
if 'Year_Built' in df.columns:
    df['Property_Age'] = CURRENT_YEAR - df['Year_Built']
    df = df.drop(columns=['Year_Built'])

# Create price per sqft feature (useful for analysis)
df['Price_Per_SqFt'] = df['Price_in_Lakhs'] / df['Size_in_SqFt']

# Create floor ratio feature
if 'Floor_No' in df.columns and 'Total_Floors' in df.columns:
    df['Floor_Ratio'] = df['Floor_No'] / df['Total_Floors'].replace(0, 1)

# Binary encode Yes/No columns
binary_cols = ['Parking_Space', 'Security']
for col in binary_cols:
    if col in df.columns:
        df[col] = df[col].map({'Yes': 1, 'No': 0}).fillna(0)

# Encode categorical columns with ordering
if 'Public_Transport_Accessibility' in df.columns:
    df['Public_Transport_Accessibility'] = df['Public_Transport_Accessibility'].map({
        'High': 3, 'Medium': 2, 'Low': 1
    }).fillna(1)

if 'Furnished_Status' in df.columns:
    df['Furnished_Status'] = df['Furnished_Status'].map({
        'Furnished': 3, 'Semi-furnished': 2, 'Unfurnished': 1
    }).fillna(1)

print("Feature engineering completed!")
print(f"New features created: Property_Age, Price_Per_SqFt, Floor_Ratio")
print(f"Property Age range: {df['Property_Age'].min()} to {df['Property_Age'].max()} years")

## 5. Remove Outliers

In [ ]:
# Remove outliers using IQR method
Q1 = df['Price_in_Lakhs'].quantile(0.25)
Q3 = df['Price_in_Lakhs'].quantile(0.75)
IQR = Q3 - Q1

lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

df = df[(df['Price_in_Lakhs'] >= lower) & (df['Price_in_Lakhs'] <= upper)]

# Remove anomalies (large houses with very low prices)
df = df[~((df['Size_in_SqFt'] > 2500) & (df['Price_in_Lakhs'] < 20))]

print(f"After outlier removal: {df.shape[0]} rows")

## 6. Prepare Features and Target

In [ ]:
# Separate features and target
# First, drop Price_Per_SqFt from features (we created it but don't want to use it for prediction)
X = df.drop(columns=['Price_in_Lakhs', 'Price_Per_SqFt'])
y = df['Price_in_Lakhs']

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"\nFeatures being used:")
print(X.columns.tolist())

## 7. Encode Categorical Features

In [ ]:
# Identify categorical and numerical columns
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

print(f"Categorical columns ({len(categorical_cols)}): {categorical_cols}")
print(f"Numerical columns ({len(numerical_cols)}): {numerical_cols}")

# Handle complex categorical columns
# For Amenities column - count number of amenities
if 'Amenities' in X.columns:
    X['Amenities_Count'] = X['Amenities'].str.split(',').str.len()
    categorical_cols.remove('Amenities')

# Reduce high cardinality features (keep top 10 categories)
def reduce_cardinality(df, col, top_n=10):
    top_categories = df[col].value_counts().head(top_n).index
    df[col] = df[col].apply(lambda x: x if x in top_categories else 'Other')
    return df

# Apply to high cardinality columns
X_processed = X.copy()
high_cardinality_cols = ['Locality', 'City', 'State']

for col in high_cardinality_cols:
    if col in X_processed.columns and col in categorical_cols:
        original_unique = X_processed[col].nunique()
        X_processed = reduce_cardinality(X_processed, col, top_n=10)
        print(f"{col}: Reduced from {original_unique} to {X_processed[col].nunique()} categories")

# Drop Locality (too granular, City and State are enough)
if 'Locality' in X_processed.columns:
    X_processed = X_processed.drop(columns=['Locality'])
    if 'Locality' in categorical_cols:
        categorical_cols.remove('Locality')
    print("Dropped 'Locality' column (too granular)")

# One-hot encoding for remaining categorical columns
X_encoded = pd.get_dummies(X_processed, columns=categorical_cols, drop_first=True)

print(f"\nFinal encoded features: {X_encoded.shape[1]} columns")

## 8. Train-Test Split

In [ ]:
# Split data (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.2, random_state=42
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

## 9. Feature Scaling

In [ ]:
# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Feature scaling completed!")

## 10. Train Random Forest Model

In [ ]:
# Train Random Forest model
print("Training Random Forest model...")

model = RandomForestRegressor(
    n_estimators=100,
    max_depth=20,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1
)

model.fit(X_train_scaled, y_train)

print("Model training completed!")

## 11. Model Evaluation

In [ ]:
# Make predictions
y_train_pred = model.predict(X_train_scaled)
y_test_pred = model.predict(X_test_scaled)

# Calculate metrics
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
train_mae = mean_absolute_error(y_train, y_train_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)

# MAPE
mape = np.mean(np.abs((y_test - y_test_pred) / y_test)) * 100

print("="*60)
print("MODEL PERFORMANCE")
print("="*60)
print(f"Train R² Score: {train_r2:.4f}")
print(f"Test R² Score:  {test_r2:.4f}")
print(f"\nTrain RMSE: ₹{train_rmse:.2f} Lakhs")
print(f"Test RMSE:  ₹{test_rmse:.2f} Lakhs")
print(f"\nTrain MAE: ₹{train_mae:.2f} Lakhs")
print(f"Test MAE:  ₹{test_mae:.2f} Lakhs")
print(f"\nMAPE: {mape:.2f}%")
print("="*60)
print(f"\nInterpretation:")
print(f"✓ Model explains {test_r2*100:.2f}% of price variance")
print(f"✓ Average prediction error: ₹{test_mae:.2f} Lakhs")
print(f"✓ Predictions are off by ~{mape:.1f}% on average")

## 12. Feature Importance

In [ ]:
# Get feature importance
feature_importance = pd.DataFrame({
    'Feature': X_encoded.columns,
    'Importance': model.feature_importances_
}).sort_values('Importance', ascending=False).head(10)

print("\nTOP 10 MOST IMPORTANT FEATURES:")
print("="*60)
for idx, row in feature_importance.iterrows():
    print(f"{row['Feature']:40s} {row['Importance']:.4f}")
print("="*60)

## 13. Sample Predictions

In [ ]:
# Show sample predictions
sample_predictions = pd.DataFrame({
    'Actual Price (Lakhs)': y_test.values[:10],
    'Predicted Price (Lakhs)': y_test_pred[:10],
    'Difference': y_test.values[:10] - y_test_pred[:10],
    'Error %': np.abs((y_test.values[:10] - y_test_pred[:10]) / y_test.values[:10] * 100)
})

print("\nSAMPLE PREDICTIONS (First 10 Test Examples):")
print("="*80)
print(sample_predictions.to_string(index=False))
print("="*80)

## 14. Save Model with Joblib

In [ ]:
# Save model, scaler, and feature names using joblib
joblib.dump(model, 'house_price_model.joblib')
joblib.dump(scaler, 'scaler.joblib')
joblib.dump(X_encoded.columns.tolist(), 'feature_names.joblib')

print("="*60)
print("✅ MODEL SAVED SUCCESSFULLY!")
print("="*60)
print(f"Model: Random Forest Regressor")
print(f"Test R² Score: {test_r2:.4f}")
print(f"Test RMSE: ₹{test_rmse:.2f} Lakhs")
print(f"Test MAE: ₹{test_mae:.2f} Lakhs")
print(f"\nSaved files:")
print("  1. house_price_model.joblib")
print("  2. scaler.joblib")
print("  3. feature_names.joblib")
print("="*60)

## 15. Load and Test Saved Model

In [ ]:
# Load the saved model
loaded_model = joblib.load('house_price_model.joblib')
loaded_scaler = joblib.load('scaler.joblib')
loaded_features = joblib.load('feature_names.joblib')

# Test with sample data
sample_data = X_test_scaled[:5]
predictions = loaded_model.predict(sample_data)

print("="*60)
print("🔮 TESTING LOADED MODEL")
print("="*60)
for i, pred in enumerate(predictions):
    actual = y_test.values[i]
    error_pct = (abs(actual-pred)/actual*100) if actual != 0 else 0
    print(f"Sample {i+1}:")
    print(f"  Actual:    ₹{actual:.2f} Lakhs")
    print(f"  Predicted: ₹{pred:.2f} Lakhs")
    print(f"  Error:     {error_pct:.2f}%")
    print()
print("="*60)
print("✅ Model loaded and tested successfully!")

## 16. Model Summary

In [ ]:
print("="*80)
print("📊 PROJECT SUMMARY")
print("="*80)
print(f"\n📁 Dataset:")
print(f"  • Total samples: {len(df):,}")
print(f"  • Features used: {X_encoded.shape[1]}")
print(f"  • Training samples: {len(X_train):,}")
print(f"  • Test samples: {len(X_test):,}")
print(f"\n🎯 Model Performance:")
print(f"  • Algorithm: Random Forest Regressor")
print(f"  • R² Score: {test_r2:.4f} ({test_r2*100:.2f}% variance explained)")
print(f"  • RMSE: ₹{test_rmse:.2f} Lakhs")
print(f"  • MAE: ₹{test_mae:.2f} Lakhs")
print(f"  • MAPE: {mape:.2f}%")
print(f"\n💡 Key Insights:")
print(f"  • Average prediction error: ₹{test_mae:.2f} Lakhs")
print(f"  • Model is production-ready")
print(f"  • Cardinality reduced for memory efficiency")
print(f"  • Saved with joblib for faster loading")
print("\n" + "="*80)
print("🎉 PROJECT COMPLETED SUCCESSFULLY!")
print("="*80)